# SOLID

## 단일 책임 원칙
- 클래스는 하나의 구체적인 일을 담당하며, 변화해야 할 이유는 단 하나뿐이어야 한다.
- 다른 이유로 클래스를 수정해야 한다면 추상화가 잘못되어 클래스에 너무 많은 책임이 있다는 뜻이다.
- 보다 응집력 있는 추상화를 하는데 도움이 된다.
> 어떤 경우에도 여러 책임을 가진 객체를 만들어서는 안된다. (유지보수가 어려워진다)
- 클래스는 작을수록 좋다.
- 어떤 변에서는 RDBMS 정규확 개념과 유사하다.
- 클래스의 메서드는 상호 배타적이며, 사로 관련이 없어야 하고 따라서 이들은 서로 다른 책임을 가지고 있으므로 더 작은 클래스로 분해할 수 있어야 한다.


### 너무 많은 책임을 가진 클래스
- 아래 클래스는 소스에서 처리할 이벤트를 가져오고, 가져온 데이터를 파싱하고, 파싱한 이벤트를 외부로 전송한다.
- 너무 많은 일을 한다.
- 독립적 동작을 하는 메서드를 하나의 인터페이스에 정의했다는 문제.
```
class SystemMonitor:
    def load_activity(self):
        """Get the events from a source, to be processed."""

    def identify_events(self):
        """Parse the source raw data into events (domain objects)."""

    def stream_events(self):
        """Send the parsed events to an external agent."""
```

### 책임 분산

- 위 클래스를 아래 형태로 분산한다.

<img alt="KakaoTalk_Photo_2022-02-20-00-10-15" height="500" src="https://user-images.githubusercontent.com/60768642/154806693-489ac6a6-a6f0-4155-bf55-56b3598316cb.jpeg" width="500"/>

## 개방/폐쇄 원칙(Open/Close Principle)
- 확장 가능하고, 새로운 요구사항이나 도메인 변화에 잘 적응하는 코드
- 새로운 문제가 발생할 경우 새로운 것을 추가만 할 뿐 기존 코드는 그대로 유지해야 한다.
- 새로운 기능 추가할 때 기존 코드를 손보면 기존 디자인이 잘못됐다는 뜻이다.

### OCP 위반 사례
- 아래 코드는 잘 동작은 하지만 문제가 있다.
    - 지원하려는 이벤트가 늘어날수록 메서드도 커질것이다.
        - 한 가지 일만 하는 것도 아니고 한 가지 일을 제대로 하지도 못한다(이건 잘 공감이 안되는데..? 한가지 일을 그래도 해내긴 하는거 아닌가?)
    - OCP 위반이다. 수정에 닫히지 않았다.
        - 새로운 이벤트가 생기면 메서드를 수정해야 한다. elif 퍼레이드

```
class Event:
    def __init__(self, raw_data):
        self.raw_data = raw_data


class UnknownEvent(Event):
    """A type of event that cannot be identified from its data."""


class LoginEvent(Event):
    """A event representing a user that has just entered the system."""


class LogoutEvent(Event):
    """An event representing a user that has just left the system."""


class SystemMonitor:
    """Identify events that occurred in the system
    >>> l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
    >>> l1.identify_event().__class__.__name__
    'LoginEvent'
    >>> l2 = SystemMonitor({"before": {"session": 1}, "after": {"session": 0}})
    >>> l2.identify_event().__class__.__name__
    'LogoutEvent'
    >>> l3 = SystemMonitor({"before": {"session": 1}, "after": {"session": 1}})
    >>> l3.identify_event().__class__.__name__
    'UnknownEvent'
    """

    def __init__(self, event_data):
        self.event_data = event_data

    def identify_event(self):
        if (
            self.event_data["before"]["session"] == 0
            and self.event_data["after"]["session"] == 1
        ):
            return LoginEvent(self.event_data)
        elif (
            self.event_data["before"]["session"] == 1
            and self.event_data["after"]["session"] == 0
        ):
            return LogoutEvent(self.event_data)

        return UnknownEvent(self.event_data)
```

### OCP 위반 해결하기(리펙토링)
- 위 예제의 문제는 SystemMonitor 클래스가 분류하려는 구체 클래스와 직접 상호작용 한다는 점이다.
- OCP를 지키려면 추상화해야한다.
- 대안은 SystemMonitor 클래스를 추상적인 이벤트와 협력하도록 하고, 이벤트에 대응하는 개별 로직은 각 이벤트 클래스에 위임하는 것이다.
- 그런 다음 각각의 이벤트에 다형성을 가진 새로운 메서드를 추가해야 한다.
- 이 메서드는 전달되는 데이터가 해당 클래스의 타입과 일치하는지 판단하는 역할을 한다.
- 또한 기존 분류 로직을 수정하여 이 메서드를 사용해 전체 이벤트를 돌면서 검사하도록 해야 한다.

- ![asdf](https://user-images.githubusercontent.com/60768642/154822827-178a0031-0999-45e0-9da4-6e824638777e.jpeg)


```
class Event:
    def __init__(self, raw_data):
        self.raw_data = raw_data

    @staticmethod
    def meets_condition(event_data: dict):
        return False


class UnknownEvent(Event):
    """A type of event that cannot be identified from its data"""


class LoginEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"]["session"] == 0
            and event_data["after"]["session"] == 1
        )


class LogoutEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"]["session"] == 1
            and event_data["after"]["session"] == 0
        )


class SystemMonitor:
    """Identify events that occurred in the system
    >>> l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
    >>> l1.identify_event().__class__.__name__
    'LoginEvent'
    >>> l2 = SystemMonitor({"before": {"session": 1}, "after": {"session": 0}})
    >>> l2.identify_event().__class__.__name__
    'LogoutEvent'
    >>> l3 = SystemMonitor({"before": {"session": 1}, "after": {"session": 1}})
    >>> l3.identify_event().__class__.__name__
    'UnknownEvent'
    """

    def __init__(self, event_data):
        self.event_data = event_data

    def identify_event(self):
        for event_cls in Event.__subclasses__():
            try:
                if event_cls.meets_condition(self.event_data):
                    return event_cls(self.event_data)
            except KeyError:
                continue
        return UnknownEvent(self.event_data)
```

### 이벤트 시스템 확장
- 새로운 유형의 이벤트인 TransactionEvent를 추가해보자. OCP를 잘 지키면서.

```
class Event:
    def __init__(self, raw_data):
        self.raw_data = raw_data

    @staticmethod
    def meets_condition(event_data: dict):
        return False


class UnknownEvent(Event):
    """A type of event that cannot be identified from its data"""


class LoginEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"]["session"] == 0
            and event_data["after"]["session"] == 1
        )


class LogoutEvent(Event):
    @staticmethod
    def meets_condition(event_data: dict):
        return (
            event_data["before"]["session"] == 1
            and event_data["after"]["session"] == 0
        )


class TransactionEvent(Event):
    """Represents a transaction that has just occurred on the system."""

    @staticmethod
    def meets_condition(event_data: dict):
        return event_data["after"].get("transaction") is not None


class SystemMonitor:
    """Identify events that occurred in the system
    >>> l1 = SystemMonitor({"before": {"session": 0}, "after": {"session": 1}})
    >>> l1.identify_event().__class__.__name__
    'LoginEvent'
    >>> l2 = SystemMonitor({"before": {"session": 1}, "after": {"session": 0}})
    >>> l2.identify_event().__class__.__name__
    'LogoutEvent'
    >>> l3 = SystemMonitor({"before": {"session": 1}, "after": {"session": 1}})
    >>> l3.identify_event().__class__.__name__
    'UnknownEvent'
    >>> l4 = SystemMonitor({"after": {"transaction": "Tx001"}})
    >>> l4.identify_event().__class__.__name__
    'TransactionEvent'
    """

    def __init__(self, event_data):
        self.event_data = event_data

    def identify_event(self):
        for event_cls in Event.__subclasses__():
            try:
                if event_cls.meets_condition(self.event_data):
                    return event_cls(self.event_data)
            except KeyError:
                continue
        return UnknownEvent(self.event_data)
```

### OCP 정리
- OCP는 다형성의 효과적인 사용과 밀접하게 관련이 있다.
- 코드를 변경하지 않고 기능을 확장하기 위해서는 보호하려는 추상화(여기에서는 새로운 이벤트 유형)에 대해서 적절한 폐쇄를 해야 한다.
- 일부 추상화의 경우 충돌이 발생할 수 있기 때문에 모든 프로그램에서 이 원칙을 적용하는 것이 가능한 것은 아니다.

## 리스코프 치환 원칙(LSP)
- 안정성을 유지하기 위해 객체 타입이 유지해야하는 일련의 특성을 말한다.
- 어떤 클래스에서든 클라이언트는 특별한 주의를 기울이지 않고도 하위 타입을 사용할 수 있어야 한다는 것이다.
- 만약 S가 T의 하위 타입이면 T타입의 객체를 S 타입의 객체로 치환 가능해야 한다.
- 좋은 클래스는 명확하고 간결한 인터페이스를 가지고 있으며, 하위 클래스가 해당 인터페이스를 따르는 한 프로그램은 정상적으로 동작한다.
- 결론적으로 이 원칙은 계약을 통한 설계와도 관련이 있다.
- 위반시 mypy, pylint, 파이참에서 알려준다. 이 경고를 무시하지 말아라.
> LSP는 OCP에 기여한다. LSP를 잘 따르면 계층을 올바르게 확장하는데 도움된다.


## 인터페이스 분리 원칙(Inteface Segregation Principle, ISP)
- ISP는 계속 얘기했던 "작은 인터페이스"에 대한 가이드라인을 제공한다.
- 인터페이스는 객체가 노출하는 메서드의 집합이다.
- 즉, 다른 클라이언트에서 호출할 수 있는 요청들이다.
> 파이썬에서 인터페이스는 클래스 메서드의 형태를 보고 암시적으로 정의된다. 이것은 duck typing 때문이다.
> 덕타이핑은 모든 객체가 자신이 가지고 있는 메서드와 자신이 할 수 있는 일에 의해서 표현된다는 점에서 출밯한다.
> 즉 클래스의 유형, 이름, docstring, 클래스 속성 또는 인스턴스 속성에 관계없이 객체의 본질을 정의하는 것은 궁극적으로 메서드의 형태다.
> 메서드는 실제로 그 객체가 무엇인지 결정한다.
> "어떤 새가 오리처럼 걷고 오리처럼 꽥꽥 소리를 낸다면 오리여야만 한다" 는 데서 덕 타이핑이라고 불린다.
> 오랫동안 덕 파이팅은 파있너에서 인터페이스를 정의하는 유일한 방법이이었다.
> 파이썬 3에서 추상 기본 클래스 개념이 생겼다.
> 추상 기본 클래스는 파생 클래스가 구현해야 할 일부분을 기본 동작 또는 인터페이스로 정의하는 것이다.
> 이는 특정 중요 메서드가 실제로 재정의 되었는지 확인이 필요할 때 유용하며, isinstance()와 같은 메서드 기능을 재정의 하거나 확장하는 매커니즘으로도 동작한다.
> 또한 이 도뮬에는 가상 하위 클래스(virtual subclass)라는 타입을 계층구조에 등록하는 기법이 포함되어 있다.
> 이것은 오리와 같이 걷고, 오리와 같이 소리를 내는 등 오리의 새로운 기준을 추가함으로써 덕 타이핑의 개념을 조금 더 확장하는 것이다.

- 추상적으로 말하자면 ISP는 다음을 뜻한다.
- 다중 메서드를 가진 인터페이스가 있다면 매우 정확하고 구체적인 구분에 따라 더 적은 수의 메서드(가급적이면 단 하나)를 가진
여러 개의 메서드로 분할하는 것이 좋다는 것이다.
- 재사용성을 높이기 위해 가능한 작은 단위로 인터페이스를 분리한다면 응집력이 높아진다.

### 너무 많은 일을 하는 인터페이스
- 아래 클래스는 너무 많은 일을 한다.
- 만약 어떤 클래스가 json만 필요해서 이클래스를 상속받으려 하면, 쓸데없이 xml기능까지 가지게 된다.
```
class Parser:
    def from_xml(self):
        pass

    def from_json(self):
        pass
```

- 아래처럼 분리해야 좋다.

```
class XMLEventParser:
    def from_xml(self):
        pass


class JSONEventParser:
    def from_json(self):
        pass
```


### 인터페이스는 작을 수록 좋다.
- SRP와 유사하지만 주요 차이점은 ISP는 인터페이스에 대해 이야기하고 있다는 점이다.
- 따라서 이것은 행동의 추상화이다.
- 이 원칙을 준수하지 않으면 별개의 기능이 결합된 인터페이스를 만들게 된다.

### 인터페이스는 얼마나 작아야 할까?
- 작아야 한다는 것을 극단적으로 받아들이면 곤란하다.
- 응집력의 관점에서 가능한 단 한가지 일을 수행하는 작은 인터페이스여야 한다.
- 그렇다고 반드시 딱 한 가지 메서드만 있어야 한다는 뜻은 아니다.
- 앞 예시에서는 두 메서드가 관련이 없기 때문에 분리했다.
- 하지만 하나 이상의 메서드라 하더라도 적절하게 하나의 클래스에 속할 수 있다.
- 예를들어 컨텍스트 매니저의 `__enter__`, `__exit__`은 같이 있어야 유의미하다.


## 의존성 역전(DIP)
- 매우 강력한 원칙이다.
- Dependancy Inversion Principle
- DIP는 코드가 깨지거나 손상되는 취약점으로부터 보호해주는 디자인 원칙을 제시한다.
- 의존성을 역전시킨다는 것은 코드가 세부 사항, 구체적 구현에 적응하지 않고, 대신 API 같은 것에 적응하도록 하는 것이다.

> A와 B 두 객체가 상호 교류한다. A는 B의 인스턴스를 사용한다.
> 외부 라이브러리 사용, 다른 팀의 모듈 사용 등의 경우다.
> 만약 코드가 B에 크게 의존하면, B가 변경되었을 때 A도 깨질 확률이 크다.
> 이걸 방지하기 위해 의존성을 뒤집어야 한다. 즉, B가 A에 적응해야 한다.
> 즉, 인터페이스를 개발하고 코드가 B의 구체적 구현에 의존하지 않도록 해야한다.
> B는 해당 인터페이스를 준수해야 한다.

- 일반적으로 구현이 추상 컴포턴트보다 훨씬 자주 바뀐다.
- 때문에 시스템 변경, 수정, 확장이 예상되는 지점에서는 유연성 확보를 위해 추상화(인터페이스)를 사용한다.


### 의존이 강하게 걸린 경우
- 아래 예시는 DIP원칙 위반이다.
- Syslog의 개별구현인 send메서드에 의존하기 때문에, Syslog의 send 메서드가 변경되면 EventStreamer도 깨질 확률이 크다.

![111](https://user-images.githubusercontent.com/60768642/154824239-c9fdf6b3-47b1-4b5b-8185-7c0b9c7991de.jpeg)


### 의존성 뒤집기
- 아래처럼 DataTargetClient라는 인터페이스를 정의하면 문제가 해결된다.
- Syslog가 DataTargetClient인터페이스를 준수해야 할 책임이 생긴다.
- EventStreamer는 인터페이스를 안전하게 믿고 사용할 수 있다.
> 심지어 런타임 중에도 send() 메서드를 구현한 객체의 프로퍼티를 수정해도 여전히 잘 동작한다.
> 이렇게 의존성을 동적으로 제공한다고 하여 종종 의존성 주입(dependency injection)이라고 한다.


![2222](https://user-images.githubusercontent.com/60768642/154824240-519aefb3-d40a-46fa-b346-5e5fd21451cb.jpeg)

- 사실 굳이 DataTargetClient인터페이스가 없어도 잘 동작한다.
- 왜 굳이 추상 기본 클래스(인터페이스)를 정의하는 것일까? 정말 필요가 있는 일인가?
- 엄밀히 말하면 맞는 말이긴 하다. 꼭 이렇게 할 필요는 없다. 결국 프로그램은 똑같이 동작한다.
- 그러나 추상 기본 클래스를 사용하는 것은 좋은 습관이다.
- 첫 번째 장점은 덕타이핑이다. 가독성이 높아진다.
- 상속은 is-a 관계다. 따라서 Syslog는 DataTargetClient라고 말할 수 있다.
- 즉 코드 사용자는 코드를 읽고 잘 이해할 수 있다(이것이 바로 덕 타이핑이다)
- 추상기본 클래스의 사용은 필수는 아니다.
- 그러나 클린 디자인을 위해서 바람직하다.
- 파이썬이 너무 유연하여 자주 발생하는 실수를 줄여줄 수 있다.